In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from PyPDF2 import PdfReader, PdfWriter
from io import BytesIO
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

(841.68, 1190.8799999999999)


In [2]:
workbookPath = 'FullSpreadsheets\CAF+v0.1_August+6,+2024\CAF v0.1_August 6, 2024_00.50.xlsx'
savePath =  'FullSpreadsheets\CAF+v0.1_August+6,+2024\CAF v0.1_August 6, 2024_00.50 filtered.xlsx'
guttmanPath = 'BOH3\CAF+v0.1_September+25,+2024_20.31\CAF v0.1_September 25, 2024_20.31 guttman.xlsx'
guttmanPath = 'DDS2\CAF+v0.1_September+2,+2024_22.43\CAF v0.1_September 2, 2024_22.43 endo guttman 28 Aug.xlsx'
guttmanPath = 'DDS2\CAF+v0.1_September+2,+2024_22.43\CAF v0.1_September 2, 2024_22.43 endo guttman teeth standard 28 Aug.xlsx'
guttmanPath = 'FullSpreadsheets\CAF+v0.1_October+7,+2024_18.49\CAF v0.1_October 7, 2024_18.49 guttman.xlsx'
guttmanPath = 'BOH2\CAF+v0.1_October+14,+2024 few students\CAF v0.1_October 14, 2024_16.56 best guttman.xlsx'
guttmanPath = 'BOH3\CAF+v0.1_October+22,+2024_22.00\CAF v0.1_October 22, 2024_22.00 guttman.xlsx'
guttmanPath = 'FullSpreadsheets\CAF+v0.1_December+5,+2024\CAF v0.1_December 5, 2024 guttman.xlsx'
workbook, folder, file = loadWorkbook(guttmanPath)

('Loaded workbook: FullSpreadsheets\\CAF+v0.1_December+5,+2024 | CAF '
 'v0.1_December 5, 2024 guttman | .xlsx')
("Workbook sheets: ['011_COE', '012_POE', '013_LIMITED OE', "
 "'014_CONSULTATION', '022_I-O RAD', '061_VITALITY', '071_DIAGNOSTIC MODEL', "
 "'072_PHOTOS', '074_PHYSICAL MODELS', '111_CLINIC', '111_SIM', '113_CLINIC', "
 "'113_SIM', '114_CLINIC', '114_SIM', '115_CLINIC', '121_CLINIC REMIN', "
 "'121_SIM REMIN', '123_CLINIC', '123_SIM', '131_CLINIC', '141_CLINIC', "
 "'142_CLINIC', '161_CLINIC', '161_SIM', '165_CLINIC', '165_SIM', "
 "'221_CLINIC', '221_SIM', '222_CLINIC', '222_SIM', '311_CLINIC', '311_SIM', "
 "'386_CLINIC', '386_SIM', '411_CLINIC', '411_SIM', '414_CLINIC CVEK', "
 "'414_CLINIC DECIDUOUS', '414_SIM CVEK', '414_SIM DECIDUOUS', '415_SIM "
 "ACCESS', '415_SIM WORK LENGTH', '416_SIM RCT ADD', '417_SIM CONE FIT', "
 "'417_SIM ROOT FLG', '418_SIM CONE FIT', '418_SIM ROOT FLG', '511_CLINIC', "
 "'511_SIM', '512_CLINIC', '513_SIM', '514_SIM', '521_CLINIC', '521_SIM

In [17]:
# for sheet in workbook.sheetnames:
#     df = loadDfFromSheet(workbook, sheet)
#     # take dates on 28th Aug
#     date = pd.to_datetime('2024-08-28')
#     df[colDate] = pd.to_datetime(df[colDate], format = '%d/%m/%Y')
#     print(df[colDate].dtype)
#     df = df[df[colDate] == date]
#     display(df)
#     colorCols = getColorColumns(df, sheet.split('_')[0])
#     saveDf(df, f'{folder}/{file} 28 Aug.xlsx', sheet, len(colorCols))

In [3]:
studentDict= {}
for i, sheet in enumerate(workbook.sheetnames):
    code = sheet.split(' ')[0]
    df = loadDfFromSheet(workbook, sheet)
    df = df[~df[colId].isin([111111, 1111111, 0, 1, 12, 123, 1234, 12345, 123456, 1234567, 12345678])]
    # df[colClinicChoice] = 'Simulation' # to remove later
    # df[colSubject] = 'DENT90115'
    stuId = df[colId].unique()
    # remove Nan
    stuId =  [x for x in stuId if str(x) != 'nan']
    for s in stuId:
        if s not in studentDict.keys():
            studentDict[s] = {}
        subdf = df[df[colId] == s]
        if len(subdf) > 0:
            studentDict[s][sheet] = subdf

In [4]:
studentDict.keys()
print('Number of students:', len(studentDict.keys()))
for id, items in studentDict.items():
    print(id, len(items.keys()), items.keys())

Number of students: 333
1354161.0 14 dict_keys(['011_COE', '012_POE', '022_I-O RAD', '061_VITALITY', '111_CLINIC', '114_CLINIC', '115_CLINIC', '121_CLINIC REMIN', '131_CLINIC', '141_CLINIC', '161_CLINIC', '221_CLINIC', '222_CLINIC', 'LA_INFILTRATION'])
1377061.0 37 dict_keys(['011_COE', '012_POE', '013_LIMITED OE', '022_I-O RAD', '061_VITALITY', '114_CLINIC', '115_CLINIC', '121_CLINIC REMIN', '131_CLINIC', '141_CLINIC', '161_CLINIC', '221_CLINIC', '221_SIM', '222_CLINIC', '222_SIM', '311_SIM', '386_SIM', '411_SIM', '414_CLINIC CVEK', '414_CLINIC DECIDUOUS', '414_SIM DECIDUOUS', '522_CLINIC', '522_SIM', '523_SIM', '524_CLINIC', '524_SIM', '525_SIM', '531_SIM', '532_SIM', '533_SIM', '534_SIM', '577_SIM', '578_SIM', '579_BONDING', '586_SIM TOOTH PREP', '587_SIM SEPARATORS', 'LA_INFILTRATION'])
1463086.0 40 dict_keys(['011_COE', '012_POE', '013_LIMITED OE', '022_I-O RAD', '061_VITALITY', '072_PHOTOS', '114_CLINIC', '114_SIM', '115_CLINIC', '121_CLINIC REMIN', '131_CLINIC', '141_CLINIC', '1

In [6]:
listFile = 'data/2024 MDS Student List_v10.xlsx'
listDf = pd.read_excel(listFile)

# listDf = listDf[['Student ID', 'Cohort']]
print(listDf['Cohort'].unique())
display(listDf.head())
# Create Student ID: Cohort dictionary
cohortDict = {}
for i, row in listDf.iterrows():
    id = row['Student ID']
    cohort = row['Cohort']
    if '2024' in cohort:
        cohortDict[id] = cohort.split(' ')[0]

# save cohortDict to json
with open('cohortDict.json', 'w') as f:
    json.dump(cohortDict, f)

['BH-BMED' 'BH-ORHLTH' 'BH-SCI' 'BOH1 (2024)' 'BOH2 (2024)' 'BOH3 (2024)'
 'DCD (Endo)' 'DCD (Oral Med)' 'DCD (Ortho)' 'DCD (Paed)' 'DCD (Perio)'
 'DCD (Pros)' 'DCD (SND)' 'DDS1 (2024)' 'DDS2 (2024)' 'DDS2 (2023)'
 'DDS3 (2024)' 'DDS4 (2024)' 'DR-PHILMDH' 'Grad Dip (Implants)'
 'MC-BMEDSC' 'MR-PHILDSC']


,Cohort,Student ID,Group,GroupID,Title,Gender,First Name,Preferred Name,Surname,Email,Date of Birth,Mobile,Left or Right Handed,Liability Category,Religion,Reps & Role,Notes,Latest EWS Date,SSP Status,Qualification,Institute,Overall WAM
0,BH-BMED,993898,NaN,NaN,NaN,NaN,Ian,NaN,Ang,ian.ang@student.unimelb.edu.au,NaN,NaN,NaN,Commonwealth Supported Place,NaN,NaN,SY 2021 Supervisor - Dr Antonio Celentano,NaT,NaN,Bachelor of Biomedicine,UoM,70.95
1,BH-BMED,912254,NaN,NaN,NaN,NaN,Bruce,NaN,Lin,b.lin9@student.unimelb.edu.au,NaN,NaN,NaN,Commonwealth Supported Place,NaN,NaN,SY 2021 Supervisor - Dr Wenyi Li,NaT,NaN,Bachelor of Science,UoM,70.96
2,BH-BMED,1248604,NaN,NaN,NaN,NaN,Mitchell,NaN,O'Neil,mitchelloneil8@gmail.com,NaN,NaN,NaN,Commonwealth Supported Place,NaN,NaN,SY 2021 Supervisor - Drs Monica Slavin & Sophi...,NaT,NaN,Bachelor of Dental Surgery,University of Adelaide,80.00
3,BH-BMED,1240507,NaN,NaN,NaN,NaN,Patrycja,NaN,Solak,patsol1312@gmail.com,NaN,NaN,NaN,Commonwealth Supported Place,NaN,NaN,SY 2021 Supervisor - Dr Catherine Butler,NaT,NaN,Bachelor of Biomedicine,La Trobe University,86.38
4,BH-BMED,912179,NaN,NaN,NaN,NaN,Yathavan,NaN,Sriharan,syathavan@icloud.com,NaN,NaN,NaN,Commonwealth Supported Place,NaN,NaN,SY 2021 Supervisor - Dr Catherine Butler,NaT,NaN,Bachelor of Biomedical Science,La Trobe University,79.88


In [7]:
# display(cohortDict)
# remove students not in 2024 cohort
keys = list(studentDict.keys())
for key in keys:
    if key not in cohortDict.keys():
        del studentDict[key]
        print(key)
print('Number of students:', len(studentDict.keys()))
display(studentDict.keys())

Number of students: 333


dict_keys([1354161.0, 1377061.0, 1463086.0, 1462377.0, 1220151.0, 1461394.0, 1443777.0, 1452784.0, 1462155.0, 1472637.0, 1463403.0, 1463879.0, 1455126.0, 1464586.0, 1461325.0, 994883.0, 1352197.0, 1472769.0, 1473169.0, 1088656.0, 1472250.0, 1462773.0, 1472445.0, 1473664.0, 1080271.0, 1473073.0, 1443680.0, 1472645.0, 1472938.0, 1270550.0, 1213084.0, 1473013.0, 1270549.0, 1461861.0, 1452755.0, 1233586.0, 1472447.0, 996164.0, 1354025.0, 1213093.0, 1423642.0, 1419850.0, 1472336.0, 1387538.0, 1461909.0, 1460391.0, 1472568.0, 1311884.0, 1211902.0, 1472277.0, 1472266.0, 1217909.0, 1280725.0, 1461567.0, 947992.0, 1362971.0, 1473826.0, 1337571.0, 1460189.0, 1452798.0, 1472800.0, 1469829.0, 1354103.0, 1469076.0, 1472810.0, 1472946.0, 1452727.0, 1452767.0, 1081205.0, 1048480.0, 1472625.0, 1452672.0, 1461239.0, 1343049.0, 1268620.0, 1472717.0, 1460683.0, 1353787.0, 1594851.0, 1255058.0, 1354146.0, 987528.0, 1180109.0, 1462810.0, 1354147.0, 1617304.0, 1616745.0, 1352187.0, 1350539.0, 1337652.0, 135

In [9]:
listdf = pd.read_excel('data/Student ID for Kunal.xlsx')
students = listdf['Student ID'].unique()
print('Number of students:', len(students))
if len(students) > 0:
    for s in students:
        if s not in studentDict.keys():
            print(s)
            # studentDict[s] = {}
# Now the reverse, find students in the dictionary that are not in the list
print('\n')
keys = list(studentDict.keys())
for key in keys:
    if key not in students:
        print(key)

Number of students: 324


1270550.0
1180109.0
1295910.0
1472831.0
1255040.0
1518931.0
1616604.0
1616364.0
1539529.0


In [10]:
pageSize = ( 11.69 * inch, 8.27 * 2 * inch) # page size
print(pageSize)
figSize = (pageSize[0] / 100, pageSize[1] / 100)

# Define the margins
leftMargin = 0 * inch
rightMargin = 0 * inch
topMargin = 1 * inch
bottomMargin = 0 * inch

# Define the styles for the headings
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Center', alignment=1))  # Center alignment
headingStyle = ParagraphStyle('Heading1', parent=styles['Heading1'], fontSize=32, alignment=1)  # Centered
heading2Style = ParagraphStyle('Heading2', parent=styles['Heading2'], fontSize=28, alignment=1)  # Centered
subheadingStyle = ParagraphStyle('Heading2', parent=styles['Heading2'], fontSize=24, alignment=1)  # Centered
subsubheadingStyle = ParagraphStyle('Heading3', parent=styles['Heading3'], fontSize=20, alignment=1)  # Centered
normalLargeStyleLeft = ParagraphStyle('NormalLarge', parent=styles['Normal'], fontSize=18, alignment=0)  # Left aligned
normalLargeStyleCenter = ParagraphStyle('NormalLarge2', parent=styles['Normal'], fontSize=18, alignment=1)  # Center aligned
tableTextStyle = ParagraphStyle('LargeFont', parent=styles['Normal'], fontSize=13, alignment=1)
tableTextStyleSmall= ParagraphStyle('SmallFont', parent=styles['Normal'], fontSize=11, alignment=1)

Checklistcolors = {'Yes': 'blue', 'No': 'orange', 'Not Reviewed': 'lightgrey'}
# Set the colorblind-friendly palette
sns.set_palette("colorblind")



(841.68, 1190.8799999999999)


In [11]:
def custom_agg(series):
    unique_vals = series.dropna().unique()
    if len(unique_vals) == 0: # All values are NA
        return pd.NA
    if len(unique_vals) == 1: # Only one unique value 
        return unique_vals[0]
    else:
        if '1' in unique_vals:
            return '1'
        elif 1 in unique_vals:
            return 1
        elif '0' in unique_vals:
            return '0'
        elif 0 in unique_vals:
            return 0
        else:
            return pd.NA

def CEAgg(series):
    unique_vals = series.dropna().unique()
    if len(unique_vals) == 0: # All values are NA
        return pd.NA
    if len(unique_vals) == 1: # Only one unique value 
        return unique_vals[0]
    else:
        if 'Yes' in unique_vals:
            return 'Yes'
        elif 'No' in unique_vals:
            return 'No'
        else:
            return pd.NA
        
def processLabel(label):
    words = label.split()  # Split the label into words
    if len(words) >= 3:
        # Truncate the third word to its first four letters
        words = words[:2] + [words[2][:4]]
    elif len(words) == 2:
        # If there are only two words, use them as is
        words = words
    elif len(words) == 1:
        # If there is only one word, use it as is
        words = words
    return ' '.join(words)

def sortByTotal(df):
    # row wise sum of the dataframe
    df['Total'] = df.sum(axis=1)
    # sort by the row wise sum
    df.sort_values(by='Total', ascending=False, inplace=True)
    # drop the total column
    df.drop(columns=['Total'], inplace=True)

In [13]:
id = 1295910
labelItem = 'Item'
labelYes = 'Yes'
labelNo = 'No'
labelNA = 'Not Reviewed'
labelCountItem = '# forms filled'
sawPatientText = 'I saw a patient'
operatorRoleText = 'Operator'
rubricTitle = {'PS': 'Professionalism', 'CS': 'Communication', 'TS': 'Time Management', 'ES': 'Entrustment'}
rubricReferenceTableDf = pd.read_excel('data/Rubric Reference Table.xlsx')
# display(rubricReferenceTableDf)
barsPerSubplot = 6
subplotsPerPage = 6
with open(variableUtils.mcReferenceFile) as f:
    mcReferenceDict = json.load(f)

class StudentInfo:
    def __init__(self, id, locationLabel=None) -> None: # locationLabel is the label for the location of the student Simulation or Clinic
        self.id = id
        name = listDf[listDf[colId] == id]['First Name'].values[0] + ' ' + listDf[listDf[colId] == id]['Surname'].values[0]
        cohort = cohortDict[id]
        print(id, name, cohort, locationLabel)
        saveFolder = f'{folder}/{cohort}'
        os.makedirs(saveFolder, exist_ok=True)
        self.dfDict = studentDict[id]
        self.locationLabel = locationLabel

        self.doc = SimpleDocTemplate(f"{saveFolder}/{str(int(id))} - {name} ({locationLabel}).pdf", pagesize=pageSize, leftMargin=leftMargin,
                    rightMargin=rightMargin, topMargin=topMargin, bottomMargin=bottomMargin)
        self.elements = []
        self.elements.append(Paragraph(f"{name} ({str(int(id))})", headingStyle))
        self.elements.append(Spacer(1, 12))
        self.elements.append(Paragraph(f"{locationLabel.upper()}", heading2Style))
        self.elements.append(Spacer(1, 12))

    # Function to create and return a plot image as BytesIO
    def createPlotImage(self, fig):
            buf = BytesIO()
            fig.savefig(buf, format='png', bbox_inches='tight')
            buf.seek(0)
            return buf

    def addPlotImage(self, fig, ratio = None):
            plotImage = self.createPlotImage(fig)
            image = Image(plotImage)
            # print(image.drawWidth, image.drawHeight)
            
            # Resize image to fit within margins
            max_height = pageSize[1] - topMargin - bottomMargin  # Max height for Page
            max_width = pageSize[0] - leftMargin - rightMargin  # Max width for Page
            if ratio is not None:
                max_width = max_width * ratio
                max_height = max_height * ratio
            aspect_ratio = min(max_width / image.drawWidth, max_height / image.drawHeight)
            image.drawWidth *= aspect_ratio
            image.drawHeight *= aspect_ratio
            # print(image.drawWidth, image.drawHeight, aspect_ratio)
            self.elements.append(image)
            # if idx + 1 < numSubplots:
            #    self.elements.append(PageBreak())
            #self.elements.append(PageBreak())
            plt.close(fig)
    
    def createMcReferenceDf(self, code, tag):
        respectiveMCKey = [key for key in mcReferenceDict.keys() if re.match(f'{code}_MC\d+', key)]
        respectiveMCKey = [key for key in respectiveMCKey if tag in key]
        # get the dict part of mcReferenceDict based on the respectiveMCKey
        mcRef = {key: mcReferenceDict[key] for key in respectiveMCKey}
        # print(mcRef)
        # Trun mcRef into a DataFrame
        if mcRef:
            mcRefdf = pd.DataFrame(list(mcRef.items()), columns=['Abbreviation', 'Full Text'])
        
            # extract MC\d+ from Abbreviation and replace it
            if code!= '114' and code != '115':
                mcRefdf['MC'] = mcRefdf['Abbreviation'].str.extract(r'(MC\d+)')
            else:
                mcRefdf['MC'] = mcRefdf['Abbreviation'].str.extract(r'(MC\d+)')
                extracted_values = mcRefdf['Abbreviation'].str.extract(r'(H/S|U/S)', expand=False)
                # print(extracted_values)
                # Add H/S or U/S to the MC column based on what is present in the Abbreviation
                mcRefdf['MC'] = mcRefdf['MC'] + ' ' + extracted_values
            # drop the Abbreviation column
            # print(mcRefdf.columns)
            # Swap columns
            mcRefdf = mcRefdf[['MC', 'Full Text']]
            # mcRefdf.drop(columns=['Abbreviation'], inplace=True)
            # display(mcRefdf)
        
        else:
            # print(mcRef)
            mcRefdf = None
        return mcRefdf

    def plotPie(self, ax, valueCounts, title, colors=None):
        # print(valueCounts)
        # define a fixed colour palette color blind friendly
        # print(valueCounts.index)
        colorList = [colors[val] for val in valueCounts.index] if colors is not None else None
        total = valueCounts.values.sum()
        # print(total)
        def autopct(pct):
            val = int(round(pct * total / 100.0))
            return '{:.0f}%\n({v:d})'.format(pct, v=val) if pct > 0 else ''
        
        wedges, texts, autotexts = ax.pie(valueCounts, labels=valueCounts.index, autopct=autopct, startangle=90, colors=sns.color_palette() if colorList is None else colorList,
                wedgeprops={'edgecolor': 'black', 'linewidth': 0.35}, labeldistance = 0.7, pctdistance=1.15, textprops={'color': 'black', 'fontsize': 10})
        for autotext in texts:
            autotext.set_color('black')
            autotext.set_fontsize(10)
        ax.set_title(title)
        

    def plotLineSeries(self, ax, data, xcol, ycol, title, color=None):
        print(xcol, ycol, data[xcol], data[ycol])
        data.plot(x=xcol, y=ycol, kind='line', ax=ax, color=color if color is not None else sns.color_palette(), marker='o', legend=False)
        ax.set_title(title)
        ax.set_yticks(range(1, 5))
        ax.set_ylim(0, 4.5)
        ax.set_xlabel(None)
        # rotate by 90
        ax.set_xticks(range(len(data[xcol])))
        ax.set_xticklabels(data[xcol], rotation=30, fontsize=8)
        ax.set_ylabel('')

    def createSubplot(self, ax, subset, show_legend=False, colors = None, xticks = None):
     
        subset.plot(kind='bar', stacked=True, ax=ax, edgecolor='black', color=colors if colors is not None else sns.color_palette(), legend=show_legend)
        # Add the legend only if show_legend is True
        if show_legend:
            ax.legend(title='', loc='upper right', bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes)
        ax.set_xlabel('')
        ax.set_ylabel('Count')
        # desired_ticks = range(len(xticks))
        # print(desired_ticks)
        # ax.set_xticks(range(len(xticks)))
        # ax.set_xticklabels(xticks)
        # y ticks should be integers
        ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
        # ylim should start from 0
        ax.set_ylim(0, ax.get_ylim()[1] * 1.1)
        # minimum gap of y ticks is 1
        if ax.get_ylim()[1] < 5:
            ax.yaxis.set_major_locator(plt.MultipleLocator(1))
        # Show gridlines on the y-axis
        ax.grid(axis='y', linestyle='--', alpha=0.6)
        # Convert index to strings to avoid errors with re.sub
        subset.index = [str(i) for i in subset.index]
        # x tick labels, remove _ and put space
        subset.index = [processLabel(i) for i in subset.index]
        ax.set_xticklabels(subset.index, rotation=45, ha='center')

        # Check for bars with total value of zero
        for i, total in enumerate(subset.sum(axis=1)):
            if total == 0 and subset.index[i] != '':
                # Overlay 'No data available' text vertically, adjusted lower
                ax.text(i, ax.get_ylim()[0] + 0.25 * ax.get_ylim()[1], 'No data available', 
                        ha='center', va='center', rotation=90, fontsize=10, color='black', alpha=0.7)


    def addGraphs(self, countsDf, maxBars = 6, maxNSubplots = 6, nCols=2, colors = None, dynamicHeight = True, startingSubplots = None):
        df = countsDf.copy()

        # Number of rows and columns for subplots
        numSubplots = len(df) // maxBars + (1 if len(df) % maxBars > 0 else 0)
        if len(df) < maxBars:
            maxBars = len(df)
        maxRows = maxNSubplots // nCols
        if dynamicHeight:
            maxRows = int(np.ceil(min(numSubplots, maxNSubplots)/ nCols))
        # Display starting subplots number of plots on the first page
        if startingSubplots is None:
            startingSubplots = maxNSubplots
            lastPageSubplots = numSubplots % maxNSubplots
        else:
            alreadyNumPlots = 3 - startingSubplots
            if alreadyNumPlots + numSubplots <= 3:
                lastPageSubplots = (alreadyNumPlots + numSubplots)
            else:
                lastPageSubplots = abs(numSubplots - startingSubplots) % maxNSubplots
            startingSubplots = min(startingSubplots, numSubplots)

        # Loop over the DataFrame and create subplots
        # for i in range(0, numSubplots, maxNSubplots):
        i = 0
        while i < numSubplots:
            if i == 0:
                    current_subplots = startingSubplots
            else:
                    current_subplots = maxNSubplots
            # print(i, current_subplots, (figsize[0], figsize[1] / maxNSubplots * current_subplots))
            nRows = int(np.ceil(current_subplots/nCols))
            fig, axes = plt.subplots(nrows=nRows, ncols=nCols, figsize=(figSize[0], figSize[1] / maxRows * nRows))
            if current_subplots == 1:
                axes = [axes]
            else:
                axes = axes.flatten()
            
            for j in range(current_subplots):
                idx = i + j
                if idx < numSubplots:
                    start_idx = idx * maxBars
                    end_idx = start_idx + maxBars
                    subset = df.iloc[start_idx:end_idx]
                    xticks = subset.index
                    # If there are fewer than maxBars, pad with empty rows
                    if len(subset) < maxBars:
                        empty_rows = pd.DataFrame(0, index=[''] * (maxBars - len(subset)), columns=subset.columns)
                        subset = pd.concat([subset, empty_rows])
                    # Show legend only for the first subplot
                    self.createSubplot(axes[j], subset, show_legend=(i == 0 and j == 0), colors=colors, xticks=xticks)
                else:
                    axes[j].remove()
            i+=current_subplots
            # fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 1), bbox_transform=fig.transFigure)
            plt.subplots_adjust(hspace=0.4)
            # plt.suptitle('Counts for each item')
            fig.tight_layout(rect=[0, 0, 1, 1])  # Adjust layout to make room for the legend
            self.addPlotImage(fig, 0.9)
            if idx + 1 < numSubplots:
                self.elements.append(PageBreak())
            #self.elements.append(PageBreak())
            plt.close(fig)
        
        if lastPageSubplots == 0:
            lastPageSubplots = maxNSubplots
        return lastPageSubplots  

    def addMCReferenceTable(self, mcRefdf, header):
        
        if mcRefdf is None:
            self.elements.append(Paragraph("No marking checklist reference found", subsubheadingStyle))
            return
        largeFontStyle = ParagraphStyle('LargeFont', parent=styles['Normal'], fontSize=13, alignment=0)

        # Create a table with MC and Full Text columns
        data = [mcRefdf.columns.to_list()] + mcRefdf.values.tolist()
        for i in range(1, len(data)):
            data[i][1] = Paragraph(data[i][1], largeFontStyle)
            data[i][0] = Paragraph(data[i][0], largeFontStyle)
        table = Table(data, colWidths=[1.5 * inch, 9 * inch])

        # Set the style for the table text as large font
        

        table_style = TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#236DB0')),  # Header row
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#FFFFFF')),  # Header text
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Center align all cells
            ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Add border around cells
            ('ALIGN', (3, 1), (3, -1), 'LEFT'),  # Left align Reason column cells
            ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),  # Change font to bold
            ('FONTSIZE', (0, 0), (-1, -1), 14),  # Increase font size
            ('BOTTOMPADDING', (0, 0), (-1, -1), 12),  # Increase bottom padding
            ('TOPPADDING', (0, 0), (-1, -1), 12),  # Increase top padding
        ])
        table.setStyle(table_style)
        mergedElement = KeepTogether([header, Spacer(1, 12), table])
        self.elements.append(mergedElement)
        # self.elements.append(table)
        # self.elements.append(PageBreak())

        
    def createTable(self, df, title, colRatio, tableWidth = 0.9, customTextCols = [], tableTextStyle = tableTextStyle, topPadding = 12, bottomPadding = 12, cellHighlight = False, headerColor = '#9C27B0'):
        
        if df.empty:
            table = Paragraph("No data found", subsubheadingStyle)
        else:
            data = [df.columns.to_list()] + df.values.tolist()
            
            # Convert the custom text columns to paragraphs
            for i in range(1, len(data)):
                for j in customTextCols:
                    data[i][j] = Paragraph(str(data[i][j]), tableTextStyle)
            
            if colRatio is not None:
                colWidths = [ratio/sum(colRatio) * pageSize[0] * tableWidth for ratio in colRatio]
            else:
                colWidths = [1 for i in range(len(df.columns))] # Equal column widths
            # print(f'Column widths: {colWidths}')
            table = Table(data, colWidths=colWidths)
            
            table_style = TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor(headerColor)),  # Header row
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor('#FFFFFF')),  # Header text
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center align all cells
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Center align all cells
                ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Add border around cells
                ('ALIGN', (3, 1), (3, -1), 'LEFT'),  # Left align Reason column cells
                ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),  # Change font to bold
                ('FONTSIZE', (0, 0), (-1, -1), 14),  # Increase font size
                ('BOTTOMPADDING', (0, 0), (-1, -1), bottomPadding),  # Increase bottom padding
                ('TOPPADDING', (0, 0), (-1, -1), topPadding),  # Increase top padding
            ])
            table.setStyle(table_style)

        mergedElement = KeepTogether([Paragraph(title, subsubheadingStyle), Spacer(1, 12), table, Spacer(1, 12)])
        self.elements.append(mergedElement)
        # self.elements.append(Spacer(1, 12))

        # Add red colour where cell values are No
        if not cellHighlight:
            return
        for i in range(1, len(data)):
            for j in range(len(data[i])):
                if data[i][j] == 'No':
                    table.setStyle(TableStyle([('TEXTCOLOR', (j, i), (j, i), colors.red)]))

    def getDfs(self):
        self.subjectDateItemDf = pd.DataFrame(columns=[colSubject, colDate, labelItem])
        self.CEDf = pd.DataFrame(columns=[colSubject, labelItem, colDate, colCE, colCEReason])
        self.operatorCount = None if self.locationLabel == 'Simulation' else 0
        self.sawPatientCount = None if self.locationLabel == 'Simulation' else 0
        self.rubricDf = pd.DataFrame(columns=[colSubject, colDate, labelItem] + rubricQues)
        self.countsMCWiseDfDict = {}
        self.fullMCDfDict = {}
        self.mcCountsDf = pd.DataFrame(index = [item.replace('_', ' ') for item in self.dfDict.keys()], columns=[labelYes, labelNo, labelNA])
        self.overallRKCDf = pd.DataFrame()
        self.itemMCReferenceDict = {}
        for item, itemDf in self.dfDict.items():

            # itemDisplay = item.replace('_', ' ')
            code = item.split('_')[0]
            tag = item.split('_')[1]
            tag = tag.replace('-', '/')
            itemDisplay = code + ' ' + tag
            if code == '941':
                continue
            mcColumns = findMCColumns(itemDf, code)
            # print(code, tag)
            # display(itemDf)
            # print(mcColumns)
            countsClinic = itemDf[colClinicChoice].value_counts()
            highestClinic = countsClinic.idxmax()
            
            if self.locationLabel is not None:
                # print(highestClinic, label)
                if self.locationLabel == 'Simulation' and self.locationLabel != highestClinic:
                    # print('Skipping', item)
                    continue
                elif self.locationLabel == 'Clinic' and highestClinic == 'Simulation':
                    # print('Skipping', item)
                    continue

            # Aggregate if tag is SIM (Take the best value for each column for each student)
            if highestClinic == 'Simulation':
                # Step 1: Define the aggregation functions for each group of columns
                aggFuncs = {col: 'first' for col in itemDf.columns if col not in mcColumns + rubricQues + [colCE, colCEReason]}
                aggFuncs.update({col: custom_agg for col in mcColumns})
                aggFuncs.update({col: 'max' for col in rubricQues})
                aggFuncs.update({colCE: CEAgg})
                aggFuncs.update({colCEReason: lambda x: ';\n '.join(x.dropna().unique())})
                itemDf = itemDf.groupby([colDate, colId], as_index=False).agg(aggFuncs) 
            
            # Get the count of operators and patients seen if clinic
            if self.locationLabel == 'Clinic':
                operatorcount = itemDf[colRole].apply(lambda x: 1 if x == operatorRoleText else 0).sum()
                # print(itemDisplay, operatorcount, len(itemDf))
                self.operatorCount += operatorcount

                sawPatientcount = itemDf[colPatient].apply(lambda x: 1 if x == sawPatientText else 0).sum()
                # print(itemDisplay, sawPatientcount, len(itemDf))
                self.sawPatientCount += sawPatientcount

            # create a df to store counts of Yes, No and Not Reviewed for each MC column
            countsMCWiseDf = pd.DataFrame(index=mcColumns, columns=[labelYes, labelNo, labelNA])
            for col in mcColumns:
                countsMCWiseDf.at[col, labelYes] = (itemDf[col] == 1).sum() + (itemDf[col] == '1').sum()
                countsMCWiseDf.at[col, labelNo] = (itemDf[col] == 0).sum() + (itemDf[col] == '0').sum()
                countsMCWiseDf.at[col, labelNA] = (itemDf[col] == 'NA').sum()
            # sort the columns based on the MC number
            countsMCWiseDf['_sort_key'] = countsMCWiseDf.index # sort based on MC column
            countsMCWiseDf['_sort_key'] = countsMCWiseDf['_sort_key'].str.extract(r'MC(\d+)').astype(int) # extract digit
            countsMCWiseDf.sort_values(by='_sort_key', inplace=True)
            countsMCWiseDf.drop(columns=['_sort_key'], inplace=True)   
            self.countsMCWiseDfDict[itemDisplay] = countsMCWiseDf

            # store the overall counts in the main df
            self.mcCountsDf.at[itemDisplay, labelYes] = countsMCWiseDf[labelYes].sum()
            self.mcCountsDf.at[itemDisplay, labelNo] = countsMCWiseDf[labelNo].sum()
            self.mcCountsDf.at[itemDisplay, labelNA] = countsMCWiseDf[labelNA].sum()

            # Get full MC df
            sort_keys = [int(re.search(r'MC(\d+)', col_name).group(1)) for col_name in mcColumns]
            # print(sort_keys)
            # Sort the columns based on the extracted numeric values
            sortedColumns = [mcColumns[i] for i in np.argsort(sort_keys)]
            fullMCDf = itemDf[[colDate]+sortedColumns]
            fullMCDf.sort_values(by=colDate, inplace=True) 
            # sort columns based on MC number
            self.fullMCDfDict[itemDisplay] = fullMCDf
            
            # Get the MC reference table
            referenceDf = self.createMcReferenceDf(code, tag)
            self.itemMCReferenceDict[itemDisplay] = referenceDf

            for i, row in itemDf.iterrows():
                # print(row[colFinished])
                if row[colFinished] == 0:
                    continue
                new_row = pd.DataFrame({colSubject: [row[colSubject]], colDate: [row[colDate]], labelItem: [itemDisplay]})
                # print(new_row)
                self.subjectDateItemDf = pd.concat([self.subjectDateItemDf, new_row], ignore_index=True)
                
                ceRow = pd.DataFrame({colSubject: [row[colSubject]], labelItem: [itemDisplay], colDate: [row[colDate]], colCE: [row[colCE]], colCEReason: [row[colCEReason]]})
                self.CEDf = pd.concat([self.CEDf, ceRow], ignore_index=True)

                rubricRowDict1 = {colSubject: [row[colSubject]], colDate: [row[colDate]], labelItem: [itemDisplay]}
                rubricRowDict2 = {col: [row[col]] for col in rubricQues}
                rubricRowDict1.update(rubricRowDict2)
                rubricRow = pd.DataFrame(rubricRowDict1)
                self.rubricDf = pd.concat([self.rubricDf, rubricRow], ignore_index=True)

        # Group by Subject and Date, then aggregate the items
        # self.groupedDf = self.subjectDateItemDf.groupby(['Subject', 'Date'])['Item'].apply(lambda x: ', '.join(sorted(set(x)))).reset_index()
        self.groupedDf = self.subjectDateItemDf.groupby([colDate])[labelItem].apply(lambda x: ', '.join(sorted(set(x)))).reset_index()
        self.ceGroupedDf = self.CEDf.groupby([colSubject, colDate, colCE, colCEReason])[labelItem].apply(lambda x: ', '.join(sorted(set(x)))).reset_index()

        for col in rubricQues:
            self.rubricDf[col] = self.rubricDf[col].astype('Int64')

        self.rubricGroupedDf = self.rubricDf.groupby([colSubject, colDate] + rubricQues)[labelItem].apply(lambda x: ', '.join(sorted(set(x)))).reset_index() # group by subject, date and rubric questions
        
        # groupo by date for a time series plot
        aggDict1 = {labelItem: ','.join}
        aggDict2 = {col: 'mean' for col in rubricQues}
        aggDict1.update(aggDict2)
        self.rubricGroupedDfDate = self.rubricGroupedDf.groupby([colDate]).agg(aggDict1).reset_index()
        self.rubricGroupedDfDate.sort_values(colDate, inplace=True)

        # group rubric by item for a average score comparison
        self.rubricGroupedDfItem = self.rubricDf.groupby([labelItem])[rubricQues].agg('mean').reset_index()
        
        # mc counts df drop na rows 
        self.mcCountsDf.dropna(inplace=True)
        sortByTotal(self.mcCountsDf)
        # display(self.subjectDateItemDf)
        # display(self.groupedDf)
        # display(self.CEDf)
        # display(self.ceGroupedDf)
        # display(self.rubricDf)
        # display(self.rubricGroupedDf)
        # display(self.rubricGroupedDfDate)
        # display(self.rubricGroupedDfItem)
        # display(self.mcCountsDf)

    def reIndexMcCountsDf(self, code, countsMCWiseDf):
        if code!= '114' and code!='115':
            countsMCWiseDf.index = [re.search(r'MC\d+', str(i)).group() if re.search(r'MC\d+', str(i)) else i for i in countsMCWiseDf.index]
        else:
            countsMCWiseDf['newIndex'] = countsMCWiseDf.index
            countsMCWiseDf['newIndex'] = countsMCWiseDf['newIndex'].str.extract(r'(MC\d+)')

            #display(countsMCWiseDf)
            extracted_values = countsMCWiseDf.index.str.extract(r'(H/S|U/S)', expand=False)
                # print(extracted_values)
                # Add H/S or U/S to the MC column based on what is present in the Abbreviation
            countsMCWiseDf['newIndex'] = countsMCWiseDf['newIndex'] + ' ' + extracted_values
            countsMCWiseDf.set_index('newIndex', inplace=True)
    
    def renameMcColumns(self, code, mcDf):
        if code!= '114' and code!='115':    
            mcDf.columns = [re.search(r'MC\d+', str(i)).group() if re.search(r'MC\d+', str(i)) else i for i in mcDf.columns]
        else:
            extracted_values = mcDf.columns.str.extract(r'(H/S|U/S)', expand=False).dropna()
            changedCols = [re.search(r'MC\d+', str(i)).group() for i in mcDf.columns if re.search(r'MC\d+', str(i)) ]
            # print(changedCols)
            # print(extracted_values)
            unchangedCols = [i for i in mcDf.columns if not re.search(r'MC\d+', str(i))]
            # print(unchangedCols)
            changedCols = [changedCols[i] + ' ' + extracted_values[i] for i in range(len(changedCols))]
            mcDf.columns = unchangedCols + changedCols
            # print(extracted_values)
            # Add H/S or U/S to the MC column based on what is present in the Abbreviation
            # mcDf.columns = mcDf.columns + ' ' + extracted_values

    def createPDF(self):

        # Create a table with Date, and Items
        # allSubjects = self.groupedDf[colSubject].unique()
        # display(self.subjectDateItemDf)
        if self.subjectDateItemDf.empty:
            return
        # print(self.subjectDateItemDf[colSubject].value_counts())
        allSubjects = [self.subjectDateItemDf[colSubject].value_counts().idxmax()]
        self.elements.append(Paragraph('Items Performed with dates', subheadingStyle))
        self.elements.append(Spacer(1, 12))
        for subject in allSubjects:
            # tableDf = self.groupedDf[self.groupedDf[colSubject] == subject][[colDate, labelItem]]
            tableDf = self.groupedDf[[colDate, labelItem]]
            tableDf[colDate] = pd.to_datetime(tableDf[colDate], format = '%d/%m/%Y')
            tableDf.sort_values(colDate, inplace=True)
            tableDf[colDate] = tableDf[colDate].dt.strftime('%d/%m/%Y')
            # tableDf[colDate] = tableDf[colDate].astype(str)
            self.createTable(tableDf, subject, [2, 8], 0.9, [1], tableTextStyle)

            # Create a table with the count of items
            # itemCountDf = self.subjectDateItemDf[self.subjectDateItemDf[colSubject] == subject][labelItem].value_counts().reset_index()
            itemCountDf = self.subjectDateItemDf[labelItem].value_counts().reset_index()
            itemCountDf.columns = [labelItem, labelCountItem]
            itemCountDf = itemCountDf.sort_values(labelCountItem, ascending=False)
            # Add bar plot for the counts
            fig, ax = plt.subplots(figsize=(figSize[0], figSize[1] / 2))
            itemCountDf.plot(x=labelItem, y=labelCountItem, kind='bar', ax=ax, color='skyblue', legend=False)
            ax.set_title(f'Count of items completed')
            ax.set_xlabel('')
            ax.set_ylabel('Count')
            xlabels = [processLabel(i) for i in itemCountDf[labelItem]]
            ax.set_xticklabels(xlabels, rotation=90, ha='center')
            ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
            ax.set_ylim(0, ax.get_ylim()[1] * 1.1)
            if ax.get_ylim()[1] < 5:
                ax.yaxis.set_major_locator(plt.MultipleLocator(1))
            ax.grid(axis='y', linestyle='--', alpha=0.6)
            plt.tight_layout()
            self.addPlotImage(fig, 0.9)
            # self.createTable(itemCountDf, f'Count of items for {subject}', [5, 2], 0.5, [0], tableTextStyle)

        # Create a table with the CE
        yesCE = self.ceGroupedDf[self.ceGroupedDf[colCE] == 'Yes'][[colDate, labelItem, colCE, colCEReason]]
        # display(yesCE)
    
        # self.elements.append(Paragraph('Items with CE', subheadingStyle))
        self.elements.append(Spacer(1, 12))
        yesCE[colDate] = yesCE[colDate].astype(str)
        self.createTable(yesCE, 'Critical Incidents', [2, 2.7, 2.3, 7], 0.9, [0, 1, 2, 3], tableTextStyle)

        # General Information
        if self.locationLabel == 'Clinic':
            self.elements.append(Paragraph('Operator and Patient Count', subheadingStyle))
            self.elements.append(Spacer(1, 12))
            self.elements.append(Paragraph(f'Number of times Operator: {self.operatorCount}, Number of Patients Seen: {self.sawPatientCount}', normalLargeStyleCenter))
            # self.elements.append(Spacer(1, 6))
            # self.elements.append(Paragraph(f'Number of Patients Seen: {self.sawPatientCount}', normalLargeStyleCenter))

        # Overall Rubric Information
        self.elements.append(PageBreak())
        self.elements.append(Paragraph('Overall Rubric Scores', subheadingStyle))
        self.elements.append(Spacer(1, 30))

        fig, axes = plt.subplots(2, 2, figsize=(figSize[0]/1.1, figSize[1]/1.1))
        colors = {1:'#0072B2', 2: '#E69F00', 3: '#009E73', 4: '#F0E442'}
        axes = axes.flatten()
        validAxes = 0
        # display(self.rubricGroupedDf)
        # replace 0 with 1
        self.rubricGroupedDf.replace(0, 1, inplace=True)
        for i, col in enumerate(rubricQues):
            valueCounts = self.rubricGroupedDf[col].value_counts()
            valueCounts.sort_index(inplace=True, ascending=False)
            if len(valueCounts) == 0:
                continue
            self.plotPie(axes[validAxes], valueCounts, rubricTitle[col], colors)
            validAxes += 1
        
        for ax in axes[validAxes:]:
            fig.delaxes(ax)
        plt.subplots_adjust(hspace=0.2, wspace=0.5)
        plt.tight_layout()
        self.addPlotImage(fig, 0.7)

        # Add reference table
        # self.elements.append(Paragraph('Rubric Reference Table', subsubheadingStyle))
        self.createTable(rubricReferenceTableDf, 'Rubric Reference Table', [1, 4, 4, 4, 4], 0.95, [0, 1, 2, 3, 4], tableTextStyleSmall, 6, 6)
        

        # Add date series for each of the rubric questions (average for the date)
        self.elements.append(PageBreak())
        self.elements.append(Paragraph('Rubric Scores Over Time', subheadingStyle))
        self.elements.append(Spacer(1, 30))

        fig, axes = plt.subplots(4, 1, figsize=(figSize[0]/1.0, figSize[1]/1.0))
        axes = axes.flatten()
        validAxes = 0
        for i, col in enumerate(rubricQues):
            data = self.rubricGroupedDfDate[[colDate, col]].dropna()
            if data.empty:
                continue
            data[colDate] = data[colDate].astype(str)
            print(data)
            self.plotLineSeries(axes[validAxes], data, colDate, col, rubricTitle[col], colors[i+1])
            validAxes += 1
        
        for ax in axes[validAxes:]:
            fig.delaxes(ax)
        
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.5, wspace=0.5)
        self.addPlotImage(fig, 0.9)

        # Add average rubric scores for each item
        self.elements.append(PageBreak())
        self.elements.append(Paragraph('Average Rubric Scores for Each Item', subheadingStyle))
        self.elements.append(Spacer(1, 30))

        fig, axes = plt.subplots(4, 1, figsize=(figSize[0]/1.2, figSize[1]/1.2))
        axes = axes.flatten()
        validAxes = 0
        for i, col in enumerate(rubricQues):
            data = self.rubricGroupedDfItem[[labelItem, col]].dropna()
            data.sort_values(col, inplace=True, ascending=False)
            if data.empty:
                continue
            # plot a bar plot
            ax = axes[validAxes]
            data.plot(x=labelItem, y=col, kind='bar', ax=axes[validAxes], color=colors[i+1], legend=False)
            ax.set_title(rubricTitle[col])
            ax.set_yticks(range(1, 5))
            ax.set_ylim(0, 4.5)
            newLables = [processLabel(label) for label in data[labelItem]]
            ax.set_xticklabels(newLables, rotation=45, ha='center', fontsize=8)
            # ax.set_xlabel(labelItem)
            ax.set_ylabel('')
            validAxes += 1

        for ax in axes[validAxes:]:
            fig.delaxes(ax)
        
        plt.tight_layout()
        # plt.subplots_adjust(hspace=0.5, wspace=0.5)
        self.addPlotImage(fig, 0.9)
       

        # Now time for MC plots
        self.elements.append(PageBreak())
        self.elements.append(Paragraph('Overall Marking Checklist Counts', subheadingStyle))
        self.elements.append(Spacer(1, 30))
        self.addGraphs(self.mcCountsDf, 6, 6, 2, colors = Checklistcolors)

        # Go itemwise
        for itemDisplay, countsMCWiseDf in self.countsMCWiseDfDict.items():
            # if all the values are zero, skip the plot
            if countsMCWiseDf.sum().sum() == 0:
                continue
            code = itemDisplay.split(' ')[0]
            self.reIndexMcCountsDf(code, countsMCWiseDf)
            self.elements.append(PageBreak())
            fullMCDf = self.fullMCDfDict[itemDisplay]
            fullMCDf.set_index(colDate, inplace=True)
            # remove rows with all values as NA
            fullMCDf.dropna(how='all', inplace=True)
            fullMCDf.reset_index(inplace=True)

            # If the maximum rowwise total of Yes, No and NA is less than equal to two, skip the plot and make table
            if len(fullMCDf.index) > 2 or len(countsMCWiseDf.index) > 14:
                self.elements.append(Paragraph(f'Marking Checklist Counts for {itemDisplay}', subheadingStyle))
                self.elements.append(Spacer(1, 30))
                self.addGraphs(countsMCWiseDf, 14, 3, 1, colors = Checklistcolors, dynamicHeight = False)

            else:
                # Full MC df
                
                self.renameMcColumns(code, fullMCDf)
                # print(fullMCDf.columns)
                fullMCDf[colDate] = fullMCDf[colDate].astype(str)
                fullMCDf.replace({1: 'Yes', 0: 'No', 'NA': 'NA', np.nan: ''}, inplace=True)
                # display(fullMCDf)
                # split fullMCDf with max 14 columns in each table
                # Maximum number of columns per table
                max_columns = 18

                # Initialize list to store DataFrame chunks
                df_chunks = []
                self.elements.append(Paragraph(f'Marking Checklist for {itemDisplay}', subheadingStyle))
                self.elements.append(Spacer(1, 30))
                # Split the DataFrame into chunks
                for i in range(0, len(fullMCDf.columns), max_columns):
                    # Select a chunk of columns
                    df_chunk = fullMCDf.iloc[:, i:i + max_columns]
                    
                    # Add the chunk to the list
                    df_chunks.append(df_chunk)

                # Create tables for each chunk
                for index, df_chunk in enumerate(df_chunks):
                    # Adjust colRatio for the current chunk
                    colRatio = [2] + [1 for _ in range(len(df_chunk.columns) - 1)]
                    
                    # Determine the title for the table
                    if index == 0:
                        title = ''
                    else:
                        title = ''  # No title for subsequent chunks
                    
                    # Create a table for the current chunk
                    self.createTable(
                        df_chunk, 
                        title, colRatio, 0.9, [0], tableTextStyle, 6, 6, cellHighlight=True
                    )
                # break
            # Add the MC reference table
            header = f"Marking Checklist Reference for {itemDisplay}"
            header = Paragraph(header, subsubheadingStyle)
            self.addMCReferenceTable(self.itemMCReferenceDict[itemDisplay], header)
        self.doc.build(self.elements)
    

id = 1637313
student = StudentInfo(id, 'Simulation')
student.getDfs()
student.createPDF()

student = StudentInfo(id, 'Clinic')
student.getDfs()
student.createPDF()

# id =1295910
# student = StudentInfo(id, 'Simulation')
# student.getDfs()
# student.createPDF()

# student = StudentInfo(id, 'Clinic')
# student.getDfs()
# student.createPDF()

1637313 Moe Myint Tha BOH2 Simulation
MC search pattern: 011_MC\d+
MC search pattern: 071_MC\d+
MC search pattern: 111_MC\d+
MC search pattern: 114_MC\d+
MC search pattern: 114_MC\d+
MC search pattern: 131_MC\d+
MC search pattern: 161_MC\d+
MC search pattern: 161_MC\d+
MC search pattern: 221_MC\d+
MC search pattern: 221_MC\d+
MC search pattern: 222_MC\d+
MC search pattern: 513_MC\d+
MC search pattern: 521_MC\d+
MC search pattern: 531_MC\d+
MC search pattern: 532_MC\d+
          Date   PS
0   01/10/2024  2.0
1   02/10/2024  1.0
2   04/06/2024  1.0
3   07/05/2024  2.0
4   07/08/2024  2.0
5   09/07/2024  2.0
6   09/10/2024  2.0
7   10/07/2024  2.0
8   10/09/2024  2.0
9   11/09/2024  2.0
10  14/05/2024  2.0
11  14/08/2024  2.0
12  15/05/2024  2.0
13  16/07/2024  2.0
14  17/04/2024  2.0
15  17/07/2024  2.0
16  17/09/2024  2.0
17  18/09/2024  2.0
18  21/05/2024  2.0
19  22/05/2024  2.0
20  23/04/2024  2.0
21  23/07/2024  2.0
22  24/04/2024  2.0
23  24/07/2024  2.0
24  24/09/2024  2.0
25  25/

In [14]:
completed=[]

In [15]:
for id in studentDict.keys():
    if id in completed:
        continue
    student = StudentInfo(id, 'Simulation')
    student.getDfs()
    student.createPDF()

    student = StudentInfo(id, 'Clinic')
    student.getDfs()
    student.createPDF()
    print(f'Completed {id}')
    completed.append(id)
    with open('students done.npy', 'wb') as f:
        np.save(f, completed)
    # break

1354161.0 Sneha Reju BOH3 Simulation
MC search pattern: 011_MC\d+
MC search pattern: 012_MC\d+
MC search pattern: 022_MC\d+
MC search pattern: 061_MC\d+
MC search pattern: 111_MC\d+
MC search pattern: 114_MC\d+
MC search pattern: 115_MC\d+
MC search pattern: 121_MC\d+
MC search pattern: 131_MC\d+
MC search pattern: 141_MC\d+
MC search pattern: 161_MC\d+
MC search pattern: 221_MC\d+
MC search pattern: 222_MC\d+
MC search pattern: LA_MC\d+
1354161.0 Sneha Reju BOH3 Clinic
MC search pattern: 011_MC\d+
MC search pattern: 012_MC\d+
MC search pattern: 022_MC\d+
MC search pattern: 061_MC\d+
MC search pattern: 111_MC\d+
MC search pattern: 114_MC\d+
MC search pattern: 115_MC\d+
MC search pattern: 121_MC\d+
MC search pattern: 131_MC\d+
MC search pattern: 141_MC\d+
MC search pattern: 161_MC\d+
MC search pattern: 221_MC\d+
MC search pattern: 222_MC\d+
MC search pattern: LA_MC\d+
          Date        PS
0   06/05/2024       3.0
1   08/04/2024       3.0
2   08/05/2024       3.0
3   09/05/2024      

## Use Later Code

#### Dynamically add element

In [16]:
# def addElement(self, element):
#     # Estimate the height of the element
#     element_height = self.estimateElementHeight(element)

#     # Check if the element fits in the remaining space
#     if self.y_pos - element_height < bottomMargin:
#         self.elements.append(PageBreak())
#         self.y_pos = self.page_height - topMargin  # Reset position for new page

#     # Add the element and adjust the remaining space
#     self.elements.append(element)
#     self.y_pos -= element_height

# def estimateElementHeight(self, element):
#     if isinstance(element, Spacer):
#         return element.height
#     elif isinstance(element, Paragraph):
#         return element.wrap(self.page_width - leftMargin - rightMargin, self.page_height)[1]
#     elif isinstance(element, Table):
#         # Wrap the table to get its height
#         return element.wrap(self.page_width - leftMargin - rightMargin, self.page_height)[1]
#     else:
#         return 0  # Default case, can be expanded based on element type